In [1]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Conv2D
from keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import sklearn
import imageio
from skimage.io import imread
from skimage.transform import resize
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import math 
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [18]:
!mkdir -p drive
!google-drive-ocamlfuse drive # 此时colab中出现drive的文件夹，里面就是你的google drive的根目录文件
# Load the images and their labels into two lists
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive") 

/bin/bash: google-drive-ocamlfuse: command not found
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load and preprocessing the dataset

In [3]:
# Loads labels
df = pd.read_csv("seedling_labels.csv")
# df["average_expert"] = (df["Expert 1"] + df["Expert 2"]  + df["Expert 3"] + df["Expert 4"]) / 4

In [5]:
images = []
labels = []
for i in range(len(df)):
    color_cam_path = df.loc[i, 'color_cam_path']
    
    label = 1 if df.loc[i, 'Expert 1'] > 2 else 0
    
    image_color = imageio.imread(color_cam_path, as_gray=False) 
    image_color_resized = cv2.resize(image_color, (180, 180)) # Resize the image to a consistent size
    # image_color_resized = cv2.resize(image_color, (180, 180))/255 # normalize the figure

    images.append(image_color_resized)
    labels.append(label)

<ipython-input-5-3438aac86d2d>:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_color = imageio.imread(color_cam_path, as_gray=False)


## Train

In [6]:
# Split the data into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(nor_images, nor_labels, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Convert the lists to tensors
train_images = tf.convert_to_tensor(train_images)
train_labels = tf.convert_to_tensor(train_labels)
val_images = tf.convert_to_tensor(val_images)
val_labels = tf.convert_to_tensor(val_labels)
test_images = tf.convert_to_tensor(test_images)
test_labels = tf.convert_to_tensor(test_labels)

# Create datasets from the tensors
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# Shuffle and batch the datasets
batch_size = 16
train_ds = train_ds.shuffle(buffer_size=len(train_images)).batch(batch_size)
val_ds = val_ds.shuffle(buffer_size=len(val_images)).batch(batch_size)
test_ds = test_ds.shuffle(buffer_size=len(test_images)).batch(batch_size)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(180, 180, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

class_weight = {0: 1.0, 1: 2.0}
model.fit(train_ds, epochs=11, class_weight=class_weight, verbose=1, validation_data=val_ds)

In [8]:
loss, accuracy = model.evaluate(test_ds, verbose=1)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

13/13 [==============================] - 2s 136ms/step - loss: 0.4208 - accuracy: 0.8090
Test loss: 0.42075392603874207, Test accuracy: 0.8090452551841736


In [14]:
# model.save('expert_ave_color_model.h5')

In [9]:
!mkdir -p saved_model
model.save('saved_model/expert_1_color_model.h5') 

### Model Check

In [12]:
# model check
all_train_data = tf.convert_to_tensor(nor_images)

In [14]:
pre_1 = model.predict(all_train_data, verbose=1)

pre_list_1 = turn_bin(pre_1)

label_1 = [] # conclusion of expert 1
for i in range(len(df)):
  label_1.append(1 if df.loc[i, 'Expert 1'] > 2 else 0)

32/32 [==============================] - 12s 365ms/step


In [16]:
acc_single_1 = acc(pre_list_1, label_1)
acc_single_1

0.852112676056338

In [20]:
# previous model check
!mkdir -p drive
!google-drive-ocamlfuse drive # 此时colab中出现drive的文件夹，里面就是你的google drive的根目录文件
# Load the images and their labels into two lists
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/MyDrive/saved_model/") 

loaded_model2 = tf.keras.models.load_model('expert_2_color_model.h5')
all_train_data = tf.convert_to_tensor(images)

pre_2 = loaded_model2.predict(all_train_data, verbose=1)
pre_list_2 = turn_bin(pre_2)

label_2 = [] # conclusion of expert 2
for i in range(len(df)):
  label_2.append(1 if df.loc[i, 'Expert 2'] > 2 else 0)

/bin/bash: google-drive-ocamlfuse: command not found
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
32/32 [==============================] - 10s 299ms/step


In [21]:
acc_single_2 = acc(pre_list_2, label_2)
acc_single_2

0.9265593561368209

# High-level data fusion

# Model Check

In [15]:
model.save('expert4_color_model.h5')

In [16]:
loaded_model = tf.keras.models.load_model('expert4_color_model.h5')

In [17]:
# Get the true labels and predicted labels for the test dataset
y_test = np.concatenate([y for x, y in test_ds], axis=0)
y_pred = np.round(model.predict(test_ds))

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

13/13 [==============================] - 4s 257ms/step
[[139   0]
 [ 60   0]]


In [18]:
model1_predictions = model.predict(test_ds)
model1_predicted_labels = np.array((model1_predictions > 0.6).astype(int)).flatten()

13/13 [==============================] - 3s 200ms/step


In [19]:
# Get the actual labels of the dataset
actual_labels_ds = test_ds.map(lambda x, y: y)
actual_labels = np.concatenate(list(actual_labels_ds.as_numpy_iterator()))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [20]:
correct_predictions = 0
total_predictions = 0
for actual_label, balanced_predicted_label in zip(actual_labels, model1_predicted_labels ):
    if actual_label == balanced_predicted_label:
        correct_predictions += 1
    total_predictions += 1


accuracy = correct_predictions / total_predictions
print("The overall accurancy:", accuracy)

The overall accurancy: 0.6984924623115578


# Load all the models

In [24]:
!mkdir -p drive
!google-drive-ocamlfuse drive # 此时colab中出现drive的文件夹，里面就是你的google drive的根目录文件
# Load the images and their labels into two lists
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/MyDrive/saved_model/") 

/bin/bash: google-drive-ocamlfuse: command not found
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
loaded_model1 = tf.keras.models.load_model('expert_1_color_model.h5')
loaded_model2 = tf.keras.models.load_model('expert_2_color_model.h5')
loaded_model3 = tf.keras.models.load_model('expert_3_color_model.h5')
loaded_model4 = tf.keras.models.load_model('expert_4_color_model.h5')

### Single model

In [28]:
all_train_data = tf.convert_to_tensor(images)

In [29]:
pre_1 = loaded_model1.predict(all_train_data, verbose=1)
pre_2 = loaded_model2.predict(all_train_data, verbose=1)
pre_3 = loaded_model3.predict(all_train_data, verbose=1)
pre_4 = loaded_model4.predict(all_train_data, verbose=1)

32/32 [==============================] - 8s 254ms/step


In [13]:
def turn_bin(data_array):
  list_bin = []
  for i in range(len(data_array)):
    if data_array[i][0] >= 0.5:
      list_bin.append(1)
    else:
      list_bin.append(0)
  return list_bin

def acc(pre_list, test_list):
  un_match = 0
  total = len(pre_list)
  for i in range(len(pre_list)):
    if pre_list[i] != test_list[i]:
      un_match += 1
  wrong_per = un_match / total
  return(1-wrong_per)

In [30]:
pre_list_1 = turn_bin(pre_1)
pre_list_2 = turn_bin(pre_2)
pre_list_3 = turn_bin(pre_3)
pre_list_4 = turn_bin(pre_4)

In [31]:
label_1 = [] # conclusion of expert 1
label_2 = []
label_3 = []
label_4 = []
for i in range(len(df)):
  label_1.append(1 if df.loc[i, 'Expert 1'] > 2 else 0)
  label_2.append(1 if df.loc[i, 'Expert 2'] > 2 else 0)
  label_3.append(1 if df.loc[i, 'Expert 3'] > 2 else 0)
  label_4.append(1 if df.loc[i, 'Expert 4'] > 2 else 0)

In [32]:
# check acc
acc_single_1 = acc(pre_list_1, label_1)
acc_single_2 = acc(pre_list_2, label_2)
acc_single_3 = acc(pre_list_3, label_3)
acc_single_4 = acc(pre_list_4, label_4)

acc_single_1, acc_single_2, acc_single_3, acc_single_4

(0.9175050301810865,
 0.9265593561368209,
 0.9356136820925554,
 0.9366197183098591)

In [33]:
label_true = []
for i in range(len(label_4)):
  if label_1[i] + label_2[i] + label_3[i] + label_4[i] > 2:
    label_true.append(1) # normal
  elif label_1[i] + label_2[i] + label_3[i] + label_4[i] < 2:
    label_true.append(0) # abnormal
  else:
    label_true.append(3)

## Majory Voting

In [72]:
def major_vote(test_list, pre_list_1, pre_list_2, pre_list_3, pre_list_4):
  wrong_num = 0
  un_decide = 0
  true_undecide = 0
  for i in range(len(test_list)):
    if pre_list_1[i] + pre_list_2[i] + pre_list_3[i] + pre_list_4[i] > 2: # more normal case
      now_pre = 1
      if test_list[i] != now_pre:
        wrong_num += 1
    if pre_list_1[i] + pre_list_2[i] + pre_list_3[i] + pre_list_4[i] < 2:
      now_pre = 0
      if test_list[i] != now_pre:
        wrong_num += 1
    if pre_list_1[i] + pre_list_2[i] + pre_list_3[i] + pre_list_4[i] == 2:
      un_decide += 1
      if test_list[i] == 3:
        true_undecide += 1
    # print(now_pre)
  return (1-wrong_num / len(test_list)), un_decide / len(test_list), true_undecide

In [73]:
un_decide_index = []
for i in range(len(label_true)):
  if label_true[i] == 3:
    un_decide_index.append(i)
len(un_decide_index)

23

In [74]:
major_vote(label_true, pre_list_1, pre_list_2, pre_list_3, pre_list_4)

(0.9527162977867203, 0.04426559356136821, 11)

In [38]:
acc_single_1, acc_single_2, acc_single_3, acc_single_4

(0.9175050301810865,
 0.9265593561368209,
 0.9356136820925554,
 0.9366197183098591)

## Weighted Voting

In [75]:
sum_acc = sum([acc_single_1, acc_single_2, acc_single_3, acc_single_4])
def wei(acc, sum_acc):
  return acc / sum_acc

wei_1 = wei(acc_single_1, sum_acc)
wei_2 = wei(acc_single_2, sum_acc)
wei_3 = wei(acc_single_3, sum_acc)
wei_4 = wei(acc_single_4, sum_acc)

In [76]:
wei_1, wei_2, wei_3, wei_4

(0.24688684353004875,
 0.24932322685435843,
 0.2517596101786681,
 0.2520303194369247)

In [77]:
def weight_vote(test_list, pre_list_1, pre_list_2, pre_list_3, pre_list_4):
  wrong_num = 0
  un_decide = 0
  for i in range(len(test_list)):
    if wei_1 * pre_list_1[i] + wei_2 * pre_list_2[i] + wei_3 * pre_list_3[i] + wei_4 * pre_list_4[i] > 0.5:
      now_pre = 1
      if test_list[i] != now_pre:
        wrong_num += 1
    if wei_1 * pre_list_1[i] + wei_2 * pre_list_2[i] + wei_3 * pre_list_3[i] + wei_4 * pre_list_4[i] < 0.5:
      now_pre = 0
      if test_list[i] != now_pre:
        wrong_num += 1
    if wei_1 * pre_list_1[i] + wei_2 * pre_list_2[i] + wei_3 * pre_list_3[i] + wei_4 * pre_list_4[i] == 0.5:
      un_decide += 1
  return (1- wrong_num / len(test_list)), un_decide / len(test_list)

In [78]:
weight_vote(label_true, pre_list_1, pre_list_2, pre_list_3, pre_list_4)

(0.9255533199195171, 0.0)

## Bayesian Consensus

In [83]:
# combine the prediction results of three methods
y_pred_single = {'exp1':pre_list_1, 'exp2':pre_list_2,
                 'exp3':pre_list_3, "exp4":pre_list_4}

In [81]:
def get_model_num(model_num):
  # model_num is i-th method (from 0)
  model_list = ['exp1', 'exp2', 'exp3', 'exp4']
  # model_list = ['svm', 'dt', 'lr']
  return model_list[model_num] # return the name of model

def cal_p(model_num, cal_val, p_now, pred_val): 
  ## calculate the probability of p_now[0], namely 0
  # model_num will choose the model considering now (from 0)\
  # cal_val is the condition we consider, namely 0
  # p_now is the prior probability using now
  # pred_val is the predition of the model_num (about this sample) = given condition when calculating
  name_model = get_model_num(model_num)
  upper = p_now[cal_val] * prob_record[name_model][cal_val][pred_val] # key: method - true - prediction 
  lower = upper + (1 - p_now[cal_val]) * prob_record[name_model][1 - cal_val][pred_val]
  p_result = upper / lower
  return p_result

In [ ]:
### calculate likelihood prob.
prob_model_true_pred = {'exp1':{}, 'exp2':{}, 'exp3':{}, 'exp4':{}}

def cal_likelihood_prob(model_num, y_test, true_val):
  # true_value_list here is [0,1]
  model_name = get_model_num(model_num)
  # print(model_name)
  y_pred = y_pred_single[model_name]
  list_y_test = y_test #.values
  
  num_r = 0
  num_f = 0
  for i in range(len(y_pred)):
    if list_y_test[i] == true_val:
      # true a or true b
      if y_pred[i] == list_y_test[i]:
      # the predition is same as true value
        num_r += 1
      else:
        num_f += 1

  percent_r = num_r / (num_r + num_f)
  percent_f = 1 - percent_r
  return percent_r, percent_f

## Model - exp1
# predict as 0-bad, prior output should be same as the prediction
percent_p0_t0_dt, percent_p1_t0_dt = cal_likelihood_prob(model_num=0, y_test=label_true, true_val=0)
# predict as 1-good 
percent_p1_t1_dt, percent_p0_t1_dt = cal_likelihood_prob(model_num=0, y_test=label_true, true_val=1)

## Model - exp2
# predict as 0-bad
percent_p0_t0_svm, percent_p1_t0_svm = cal_likelihood_prob(model_num=1, y_test=label_true, true_val=0)
# predict as 1-good
percent_p1_t1_svm, percent_p0_t1_svm = cal_likelihood_prob(model_num=1, y_test=label_true, true_val=1)

## Model - exp3
# predict as 0-bad 
percent_p0_t0_lr, percent_p1_t0_lr = cal_likelihood_prob(model_num=2, y_test=label_true, true_val=0)
# predict as 1-good
percent_p1_t1_lr, percent_p0_t1_lr = cal_likelihood_prob(model_num=2, y_test=label_true, true_val=1)

## Model - exp4
# predict as 0-bad 
percent_p0_t0_4, percent_p1_t0_4 = cal_likelihood_prob(model_num=3, y_test=label_true, true_val=0)
# predict as 1-good
percent_p1_t1_4, percent_p0_t1_4 = cal_likelihood_prob(model_num=3, y_test=label_true, true_val=1)

In [86]:
# prob_result = {'dt_true_0':{'pred_0':percent_p0_t0_dt, 'pred_1':percent_p1_t0_dt},
#          'dt_true_1':{'pred_0':percent_p0_t1_dt, 'pred_1':percent_p1_t1_dt},
#          'svm_true_0':{'pred_0':percent_p0_t0_svm, 'pred_1':percent_p1_t0_svm},
#          'svm_true_1':{'pred_0':percent_p0_t1_svm, 'pred_1':percent_p1_t1_svm},
#          'lr_true_0':{'pred_0':percent_p0_t0_lr, 'pred_1':percent_p1_t0_lr},
#          'lr_true_1':{'pred_0':percent_p0_t1_lr, 'pred_1':percent_p1_t1_lr}}

y_test_val = label_true #.values

prob_record = {'exp1':{0:{0:percent_p0_t0_dt, 1:percent_p1_t0_dt},
         1:{0:percent_p0_t1_dt, 1:percent_p1_t1_dt}},
         'exp2':{0:{0:percent_p0_t0_svm, 1:percent_p1_t0_svm},
         1:{0:percent_p0_t1_svm, 1:percent_p1_t1_svm}},
         'exp3':{0:{0:percent_p0_t0_lr, 1:percent_p1_t0_lr},
         1:{0:percent_p0_t1_lr, 1:percent_p1_t1_lr}},
         'exp4':{0:{0:percent_p0_t0_4, 1:percent_p1_t0_4}, 
         1:{0:percent_p0_t1_4, 1:percent_p1_t1_4}}
         }
# key: method - true - prediction 

In [87]:
prob_record

{'exp1': {0: {0: 0.9681620839363242, 1: 0.03183791606367581},
  1: {0: 0.18214285714285716, 1: 0.8178571428571428}},
 'exp2': {0: {0: 0.9232995658465991, 1: 0.07670043415340089},
  1: {0: 0.07499999999999996, 1: 0.925}},
 'exp3': {0: {0: 0.9421128798842258, 1: 0.05788712011577424},
  1: {0: 0.10357142857142854, 1: 0.8964285714285715}},
 'exp4': {0: {0: 0.934876989869754, 1: 0.06512301013024602},
  1: {0: 0.05714285714285716, 1: 0.9428571428571428}}}

In [90]:
num_true0_pre0 = 0
num_true0_pred1 = 0
for i in range(len(label_true)):
  if label_true[i] == 0: # true 0
    if pre_list_1[i] == 0:
      num_true0_pre0 += 1
    else: 
      num_true0_pred1 += 1
num_true0_pre0 / (num_true0_pre0 + num_true0_pred1)

0.9681620839363242

In [92]:
diff_i = []
for i in range(len(pre_list_1)):
  if pre_list_1[i]+ pre_list_2[i]+ pre_list_3[i]+ pre_list_4[i] not in [0,4]:
    diff_i.append(i)
    # print(y_test_val[i], y_pred_dt[i], y_pred_svm[i], y_pred_lr[i])


In [ ]:
diff_i

In [97]:
num_method = 4
y_test = label_true
y_pred_bay = np.zeros(len(y_test))


for sample_i in range(len(y_test)): # len(y_test)
  p_update = {0:0.5, 1:0.5} # [0, 1]
  for num_model in range(num_method):
    name_model = get_model_num(num_model)
    p_update[0] = cal_p(model_num=num_model, cal_val=0, p_now=p_update, pred_val=y_pred_single[name_model][sample_i])
    p_update[1] = 1 - p_update[0]
    if sample_i == 10:
      print(p_update)
  # choose the predition result with higher prob.
  y_pred_bay[sample_i] = max(p_update, key=lambda x:p_update[x])

## show result
print("Accuracy of Bayesian Consensus:", metrics.accuracy_score(y_test, y_pred_bay))

{0: 0.8416568940649978, 1: 0.15834310593500223}
{0: 0.9849479385730381, 1: 0.015052061426961916}
{0: 0.9983227773769688, 1: 0.0016772226230311604}
{0: 0.9998973207667026, 1: 0.00010267923329743134}
Accuracy of Bayesian Consensus: 0.9195171026156942
